# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [7]:
df = pd.read_pickle(r"shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


## 1. Which borough has had the second highest total number of crashes reported since 2012?

In [21]:
# Count the number of crashes in each borough
borough_crashes = df['BOROUGH'].value_counts()

# Get the borough with the second highest total number of crashes
second_highest_borough = borough_crashes.nlargest(2).index[-1]

second_highest_borough

'QUEENS'

## 2. Considering the entire data set since July 2012, which borough has the most crashes for every 100,000 people?

In [22]:
# Define the population for each borough
population = pd.Series({
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
})

# Calculate the total number of crashes in each borough
borough_crashes = df['BOROUGH'].value_counts()

# Calculate the number of crashes per 100,000 people for each borough
crashes_per_capita = (borough_crashes / population) * 100000

# Get the borough with the most crashes per 100,000 people
borough_most_crashes_per_capita = crashes_per_capita.idxmax()
most_crashes = crashes_per_capita.max()

print(f"{borough_most_crashes_per_capita}, with {most_crashes:.5f} crashes since July 2012 per 100k population")

MANHATTAN, with 19121.62810 crashes since July 2012 per 100k population


## 3. For this dataset, what is the leading cause of crashes? What proportion of accidents are attributable to this cause?

In [23]:
# Remove rows where the contributing factor vehicle 1 is 'Unspecified'
df_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

# Count the number of crashes for each contributing factor
factor_counts = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# Get the contributing factor with the highest total number of crashes
leading_cause = factor_counts.idxmax()
leading_cause_count = factor_counts.max()

# Calculate the proportion of accidents attributable to this cause
total_accidents = len(df_filtered)
proportion = (leading_cause_count / total_accidents) * 100

print(f"The leading cause of crashes is {leading_cause}, which is responsible for {proportion:.2f}% of accidents.")

The leading cause of crashes is Driver Inattention/Distraction, which is responsible for 30.27% of accidents.


## 4. Continuing from the previous question - the top 5 causes of crashes account for what proportion of total crashes?

In [24]:
# Get the counts of the top 5 contributing factors
top_5_factors_counts = factor_counts.nlargest(5)

# Calculate the total number of crashes for the top 5 contributing factors
top_5_total = top_5_factors_counts.sum()

# Calculate the proportion of total crashes accounted for by the top 5 contributing factors
proportion_top_5 = (top_5_total / total_accidents) * 100

print(f"The top 5 causes of crashes account for {proportion_top_5:.2f}% of total crashes.")

The top 5 causes of crashes account for 57.76% of total crashes.


## 5. Considering the entire data since 2012, what is the total count of accidents that involved two or more fatalities?  

In [25]:
# Replace missing values in the 'NUMBER OF PERSONS KILLED' column with zero
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)

# Select only the rows where two or more people were killed
fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] >= 2]

# Get the total count of these accidents
total_fatal_accidents = len(fatal_accidents)

total_fatal_accidents

88

## 6. On average, for every 1000 accidents, how many have resulted in at least one person dead?

In [26]:
# Select only the rows where at least one person was killed
fatal_accidents = df[df['NUMBER OF PERSONS KILLED'] >= 1]

# Get the number of these accidents
num_fatal_accidents = len(fatal_accidents)

# Get the total number of accidents
total_accidents = len(df)

# Calculate the average number of fatal accidents per 1000 accidents
average_fatal_accidents = (num_fatal_accidents / total_accidents) * 1000

average_fatal_accidents

1.3893258747079764

## 7. What proportion of accidents in the data do not have a Cross Street Name?

In [27]:
# Check for missing values in the 'CROSS STREET NAME' column
missing_cross_street_name = df['CROSS STREET NAME'].isna()

# Get the total number of accidents that do not have a Cross Street Name
num_missing_cross_street_name = missing_cross_street_name.sum()

# Get the total number of accidents
total_accidents = len(df)

# Calculate the proportion of accidents that do not have a Cross Street Name
proportion_missing_cross_street_name = (num_missing_cross_street_name / total_accidents) * 100

proportion_missing_cross_street_name

37.4350983156158

## 8. The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident. Which combination of vehicles have the most number of accidents?

In [28]:
# Create a new column that combines the 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' columns
df['VEHICLE COMBINATION'] = df['VEHICLE TYPE CODE 1'] + " & " + df['VEHICLE TYPE CODE 2']

# Count the number of accidents for each combination of vehicles
vehicle_combination_counts = df['VEHICLE COMBINATION'].value_counts()

# Get the combination of vehicles with the highest total number of accidents
most_common_combination = vehicle_combination_counts.idxmax()

most_common_combination

'Sedan & Sedan'

## 9. Among crashes where the contributing factor (CONTRIBUTING FACTOR VEHICLE 1) was alcohol involvement, what proportion resulted in a fatality?

In [29]:
# Select only the rows where the contributing factor was Alcohol Involvement
alcohol_involvement = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Get the total number of these accidents
total_alcohol_involvement = len(alcohol_involvement)

# Select only the rows where at least one person was killed
fatal_alcohol_involvement = alcohol_involvement[alcohol_involvement['NUMBER OF PERSONS KILLED'] >= 1]

# Get the number of these accidents
num_fatal_alcohol_involvement = len(fatal_alcohol_involvement)

# Calculate the proportion of accidents that resulted in a fatality
proportion_fatal = (num_fatal_alcohol_involvement / total_alcohol_involvement) * 100

proportion_fatal

0.46638345031400075

## 10. What proportion of crashes occur during the evening rush hour, defined as starting at 4 PM, and before 7 PM?

In [30]:
# Convert 'CRASH DATE_CRASH TIME' to datetime format if it's not already
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])

# Select only the rows where the crash time is between 4 PM and 7 PM
rush_hour_accidents = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

# Get the number of these accidents
num_rush_hour_accidents = len(rush_hour_accidents)

# Get the total number of accidents
total_accidents = len(df)

# Calculate the proportion of accidents that occur during the evening rush hour
proportion_rush_hour = (num_rush_hour_accidents / total_accidents) * 100

proportion_rush_hour

20.514010935243242

## 11. Among crashes involving motorcycles, what proportion resulted in injuries but no fatalities?

In [31]:
# Select only the rows where a motorcycle was involved
motorcycle_accidents = df[(df['VEHICLE TYPE CODE 1'] == 'MOTORCYCLE') | (df['VEHICLE TYPE CODE 2'] == 'MOTORCYCLE')]

# Get the total number of these accidents
total_motorcycle_accidents = len(motorcycle_accidents)

# Select only the rows where there were injuries but no fatalities
injury_accidents = motorcycle_accidents[(motorcycle_accidents['NUMBER OF PERSONS INJURED'] > 0) & (motorcycle_accidents['NUMBER OF PERSONS KILLED'] == 0)]

# Get the number of these accidents
num_injury_accidents = len(injury_accidents)

# Calculate the proportion of accidents that resulted in injuries but no fatalities
proportion_injury = (num_injury_accidents / total_motorcycle_accidents) * 100

proportion_injury

50.04565018912221

## 12. How many crashes involved bicycles as one of the vehicles?

In [32]:
# Select only the rows where a bicycle was involved
bicycle_accidents = df[(df['VEHICLE TYPE CODE 1'] == 'BICYCLE') | (df['VEHICLE TYPE CODE 2'] == 'BICYCLE')]

# Get the number of these accidents
num_bicycle_accidents = len(bicycle_accidents)

num_bicycle_accidents

19108